In [35]:
import visual_behavior_glm.GLM_analysis_tools as gat

import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## get cached dropout summary results from mongo database

In [ ]:
rs = gat.retrieve_results(search_dict = {'glm_version': '6_L2_optimize_by_session'}, results_type='summary')

In [5]:
rs.columns

Index(['_id', 'dropout', 'absolute_change_from_full', 'variance_explained',
       'variance_explained_full', 'fraction_change_from_full',
       'cell_specimen_id', 'adj_fraction_change_from_full',
       'adj_variance_explained', 'adj_variance_explained_full',
       'ophys_experiment_id', 'ophys_session_id', 'behavior_session_id',
       'glm_version', 'entry_time_utc', 'container_id', 'project_code',
       'container_workflow_state', 'experiment_workflow_state', 'session_name',
       'session_type', 'equipment_name', 'date_of_acquisition',
       'isi_experiment_id', 'specimen_id', 'sex', 'age_in_days',
       'full_genotype', 'reporter_line', 'driver_line', 'imaging_depth',
       'targeted_structure', 'published_at', 'super_container_id', 'cre_line',
       'session_tags', 'failure_tags', 'exposure_number',
       'model_outputs_available', 'location', 'has_events', 'session_number'],
      dtype='object')

## identify all of the unique dropout names

In [ ]:
dropouts = rs['dropout'].unique()

## exclude 'single' and 'image' dropouts

In [43]:
dropouts_to_use = [d for d in dropouts if not d.startswith('single') and not d.startswith('image')]
dropouts_to_use

['Full',
 'all-images',
 'beh_model',
 'change',
 'correct_rejects',
 'false_alarms',
 'hits',
 'intercept',
 'misses',
 'model_bias',
 'model_omissions1',
 'model_task0',
 'model_timing1D',
 'omissions',
 'post_lick_bouts',
 'post_licks',
 'pre_lick_bouts',
 'pre_licks',
 'pupil',
 'rewards',
 'running',
 'time',
 'visual',
 'face_motion_energy']

## build a pivoted dataframe with each cell/session as a single row and 'adj_fraction_change_from_full' as the value to keep

In [44]:
rs_pivoted = gat.build_pivoted_results_summary(results_summary=rs, cutoff=0.01, value_to_use='adj_fraction_change_from_full')

In [45]:
len(rs_pivoted)

46506

## get the dominant dropout for every session/cell:

In [46]:
for idx,row in rs_pivoted.iterrows():
    rs_pivoted.at[idx,'dominant_dropout'] = rs_pivoted[dropouts_to_use].iloc[idx].abs().idxmax()
    print('idx: {}   '.format(idx), end='\r')

In [47]:
rs_pivoted.columns

Index(['identifier', 'Full', 'all-images', 'beh_model', 'change',
       'correct_rejects', 'face_motion_energy', 'false_alarms', 'hits',
       'image0', 'image1', 'image2', 'image3', 'image4', 'image5', 'image6',
       'image7', 'image_expectation', 'intercept', 'misses', 'model_bias',
       'model_omissions1', 'model_task0', 'model_timing1D', 'omissions',
       'post_lick_bouts', 'post_licks', 'pre_lick_bouts', 'pre_licks', 'pupil',
       'rewards', 'running', 'single-all-images', 'single-beh_model',
       'single-change', 'single-correct_rejects', 'single-face_motion_energy',
       'single-false_alarms', 'single-hits', 'single-image0', 'single-image1',
       'single-image2', 'single-image3', 'single-image4', 'single-image5',
       'single-image6', 'single-image7', 'single-image_expectation',
       'single-misses', 'single-model_bias', 'single-model_omissions1',
       'single-model_task0', 'single-model_timing1D', 'single-omissions',
       'single-post_lick_bouts', 'singl

## look at value counts of dominant dropouts

In [48]:
rs_pivoted.query('dominant_dropout == "face_motion_energy"').groupby(['cre_line'])['dominant_dropout'].value_counts()

cre_line           dominant_dropout  
Slc17a7-IRES2-Cre  face_motion_energy    1318
Sst-IRES-Cre       face_motion_energy      62
Vip-IRES-Cre       face_motion_energy     256
Name: dominant_dropout, dtype: int64

In [55]:
rs_pivoted.groupby(['cre_line'])['dominant_dropout'].value_counts(normalize=True)

cre_line           dominant_dropout  
Slc17a7-IRES2-Cre  visual                0.671630
                   omissions             0.087773
                   all-images            0.077623
                   running               0.053200
                   face_motion_energy    0.033955
                   pupil                 0.033414
                   correct_rejects       0.011155
                   post_lick_bouts       0.004612
                   pre_lick_bouts        0.004560
                   false_alarms          0.003839
                   post_licks            0.003504
                   beh_model             0.002885
                   rewards               0.002241
                   hits                  0.001675
                   misses                0.001675
                   time                  0.001649
                   change                0.001288
                   model_omissions1      0.000876
                   model_task0           0.000618
            

In [54]:
filepath = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020.09.24_dropout_summary_v6_opt_by_session.csv'
rs_pivoted.to_csv(filepath, index=False)